In [2]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
log=XGBClassifier()

In [16]:
params_logistic={
    "max_iter" :1000,
    "solver" :"lbfgs",
    'penalty' :'l2'
}

params_random={
  "n_estimators":150,
  "criterion" :'entropy',
  "max_depth":10,
  "min_samples_split": 10,
  "min_samples_leaf": 4,
  "bootstrap": True,
  "oob_score": True,
  "random_state": 888,
}

params_xgboost={
    'n_estimators': 100,
    'max_depth': 6,
    'learning_rate': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 888,
}



In [9]:
from imblearn.combine import SMOTETomek
smt=SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

In [14]:
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([703, 703]))

In [17]:
models=[
  (
    'LogisticRegression',
    LogisticRegression(**params_logistic),
    (X_train, y_train),
    (X_test, y_test)
  ),
  ('RandomForestClassifier',
   RandomForestClassifier(**params_random),
    (X_train, y_train),
    (X_test, y_test)
  ),
  ('XGBClassifier',
   XGBClassifier(**params_xgboost),
    (X_train_res, y_train_res),
    (X_test, y_test)
  )
]

In [18]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    report = {
        'model': model_name,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1
    }
    reports.append(report)

In [19]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

from mlflow import MlflowClient

client=MlflowClient(tracking_uri="http://127.0.0.1:8080")


In [20]:
experiment_description = (
  "This experiment is for classification models on imbalanced dataset using Logistic Regression, Random Forest, and XGBoost."
"Imbalanced_Classification_Experiment"
)

In [ ]:
experiment_tag={
  "owner": "data_scientist_team",
  "project": "Imbalanced_Classification",
  "Goal" : "Evaluate classification models on imbalanced dataset",
  "Version": "1.0",
  "Release_date": "2025-10-29",
  "mlflow_version": mlflow.__version__,
  "mlflow.note.content":experiment_description  
}

In [54]:
client.create_experiment(
    name="Imbalanced_Classification_Experiment",
    tags=experiment_tag,
)

RestException: RESOURCE_ALREADY_EXISTS: Experiment 'Imbalanced_Classification_Experiment' already exists.

In [24]:
reports[0]

{'model': 'LogisticRegression',
 'accuracy': 0.94,
 'precision': 0.8125,
 'recall': 0.5909090909090909,
 'f1_score': 0.6842105263157895}

In [35]:
para=models[1][1]
para.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 150,
 'n_jobs': None,
 'oob_score': True,
 'random_state': 888,
 'verbose': 0,
 'warm_start': False}

In [47]:
reports[0]

{'model': 'LogisticRegression',
 'accuracy': 0.94,
 'precision': 0.8125,
 'recall': 0.5909090909090909,
 'f1_score': 0.6842105263157895}

In [52]:
# Initialize MLflow
mlflow.set_experiment("Imbalanced_Classification_Experiment")
mlflow.set_tracking_uri("http://127.0.0.1:8080")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    para=element[1].get_params()
    report = reports[i]
    name=model_name + "_model"
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("params", para)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('Recall', report['recall'])
        mlflow.log_metric('Precision', report['precision'])
        mlflow.log_metric('f1_score', report['f1_score'])        
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(xgb_model=model, name=name)
        else:
            mlflow.sklearn.log_model(sk_model=model, name=name) 

2025/10/29 23:41:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LogisticRegression at: http://127.0.0.1:8080/#/experiments/867292722097676606/runs/500087c25edd4cc99fa9ebc2741305e3
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/867292722097676606


2025/10/29 23:42:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run RandomForestClassifier at: http://127.0.0.1:8080/#/experiments/867292722097676606/runs/724d52a26a264b3798d21feda836f010
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/867292722097676606


2025/10/29 23:42:18 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://127.0.0.1:8080/#/experiments/867292722097676606/runs/b29c0c45385442b7970995c66e906406
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/867292722097676606


## Register the model (By comapairing we get to know Random forest perform best so we chose this in model Registry)

In [55]:
model_name="RandomForestClassifier"
run_id=input("Enter the run ID to register the model: ")
model_uri=f"runs:/{run_id}/{model_name}_model"

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'RandomForestClassifier'.
2025/10/30 00:00:25 WARNING mlflow.tracking._model_registry.fluent: Run with id 724d52a26a264b3798d21feda836f010 has no artifacts at artifact path 'RandomForestClassifier_model', registering model based on models:/m-d62d5495ee6340c181561439cc6f13ee instead
2025/10/30 00:00:25 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 1
Created version '1' of model 'RandomForestClassifier'.


🏃 View run RandomForestClassifier at: http://127.0.0.1:8080/#/experiments/867292722097676606/runs/724d52a26a264b3798d21feda836f010
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/867292722097676606


### Load Model

In [56]:
model_version = 1
model_uri = f"models:/{model_name}/{model_version}"

loaded_model = mlflow.sklearn.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 1, 0, 0])

In [59]:
y_test[:4]

array([0, 1, 0, 0])

#### Transition to production

In [60]:
current_model_uri = f"models:/{model_name}@challenger"
production_model_name = "RandomForestClassifier_Production"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'RandomForestClassifier_Production'.
Copied version '1' of model 'RandomForestClassifier' to version '1' of model 'RandomForestClassifier_Production'.


<ModelVersion: aliases=[], creation_timestamp=1761763354107, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1761763354107, metrics=None, model_id=None, name='RandomForestClassifier_Production', params=None, run_id='724d52a26a264b3798d21feda836f010', run_link='', source='models:/RandomForestClassifier/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [65]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.sklearn.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 1, 0, 0])